<a href="https://colab.research.google.com/github/V-Nayak/DataScience_Assignments/blob/main/Assignment_18_June.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Assignment 02 (AIQoD)**

You have to create a ML model that predict the probability that a piece of
text belongs to a particular class. Use techniques like Bag of Words, tf idf vectorization and word embedding.

Please use Hash field value and
explain how you are going to use the Hash field.

## **Data extraction**
From the documents nGrams have been extracted, Each row in the
Train.csvcorresponds to one such nGram.
Features
For a given nGram several features have been extracted (145). These
features have been saved in the train.csvand test.csv. They have
parsing, spatial, content and relative information.
• Content: The cryptographic hash of the raw text.
• Parsing: nGram is a number, text, alphanumeric, etc.
• Spatial: Position and size of the nGram
• Relational: details of text nearby the nGram


In [ ]:
import pandas as pd
import numpy as np
import hashlib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Loading the Data

In [ ]:
df= pd.read_csv("/content/drive/MyDrive/Data Science Assignement/AIQoD/train.csv")

In [ ]:
df.head()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1,NO,NO,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.576561,0.073139,0.481394,0.115697,0.472474,...,0.0,0.810,3306,4676,YES,NO,YES,2,0.375535,0.464610
1,2,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.510,4678,3306,YES,NO,YES,4,0.741682,0.593630
2,3,NO,NO,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.341803,0.051422,0.935572,0.041440,0.501710,...,0.0,0.850,4678,3306,NO,NO,NO,1,0.776467,0.493159
3,4,YES,NO,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,0.653912,0.041471,0.940787,0.090851,0.556564,...,0.0,0.945,3306,4678,NO,NO,YES,3,0.168234,0.546582
4,5,NO,NO,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,1.415919,0.000000,1.000000,0.000000,0.375297,...,0.0,1.000,1263,892,NO,NO,NO,1,0.246637,0.361045


In [ ]:
df.dtypes

id        int64
x1       object
x2       object
x3       object
x4       object
         ...   
x141     object
x142     object
x143      int64
x144    float64
x145    float64
Length: 146, dtype: object

In [ ]:
df.describe()

,id,x5,x6,x7,x8,x9,x15,x16,x17,x18,...,x133,x134,x135,x136,x137,x138,x139,x143,x144,x145
count,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,...,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000
mean,5000.000000,0.964933,0.053623,0.791668,0.172705,0.448091,6.056806,0.418593,4.483448,8.073907,...,9.291529,8.800880,0.933771,0.056523,0.924781,2669.290629,2142.907091,3.667767,0.519482,0.505084
std,2886.607005,0.524688,0.128503,0.353194,0.329020,0.300549,8.756490,0.294579,4.388639,7.032278,...,7.576648,6.584598,0.069921,1.520931,0.118496,1588.220409,1386.559408,11.014722,0.270886,0.256323
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,-0.098334,-17.000000,0.000000,892.000000,838.000000,0.000000,0.021635,0.001327
25%,2500.500000,0.642796,0.000000,0.831299,0.000000,0.199617,1.000000,0.165599,2.000000,3.000000,...,4.000000,4.000000,0.916144,0.000000,0.880000,1262.000000,892.000000,1.000000,0.267939,0.306031
50%,5000.000000,1.285541,0.000000,0.960637,0.000000,0.439504,3.000000,0.395740,4.000000,7.000000,...,7.000000,7.000000,0.949552,0.000000,1.000000,1263.000000,918.000000,2.000000,0.517937,0.484561
75%,7499.500000,1.414798,0.058076,1.000000,0.151543,0.682880,7.000000,0.683908,6.000000,11.000000,...,13.000000,12.000000,0.972222,0.000000,1.000000,4672.000000,3308.000000,5.000000,0.759242,0.707324
max,9999.000000,1.474916,0.992745,1.000000,1.393289,1.098257,88.000000,1.180828,132.000000,118.000000,...,84.000000,118.000000,0.999947,23.000000,1.000000,5217.000000,6622.000000,697.000000,1.238562,1.086371


In [ ]:
df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [ ]:
# Check for NaN values
nan_counts = df.isna().sum()
print(nan_counts)

id         0
x1      1426
x2      1426
x3      1426
x4      1426
        ... 
x141       0
x142       0
x143       0
x144       0
x145       0
Length: 146, dtype: int64


In [ ]:
df.x1.unique()

array(['NO', nan, 'YES'], dtype=object)

In [ ]:
# Define a mapping dictionary for conversion
bool_map = {'YES': True, 'NO': False}

# Iterate over each column in the DataFrame
for col in df.columns:
    # Check if column contains 'yes' and 'no' values (object dtype)
    if df[col].dtype == 'object' and df[col].isin(['YES', 'NO',]).all() or df[col].dtype == 'object' and df[col].isin(['YES', 'NO',np.NaN]).all():
        # Convert 'yes'/'no' values to boolean dtype using map
        df[col] = df[col].map(bool_map).astype(bool)

df

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1,False,False,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.576561,0.073139,0.481394,0.115697,0.472474,...,0.0,0.810,3306,4676,True,False,True,2,0.375535,0.464610
1,2,True,True,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.510,4678,3306,True,False,True,4,0.741682,0.593630
2,3,False,False,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.341803,0.051422,0.935572,0.041440,0.501710,...,0.0,0.850,4678,3306,False,False,False,1,0.776467,0.493159
3,4,True,False,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,0.653912,0.041471,0.940787,0.090851,0.556564,...,0.0,0.945,3306,4678,False,False,True,3,0.168234,0.546582
4,5,False,False,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,1.415919,0.000000,1.000000,0.000000,0.375297,...,0.0,1.000,1263,892,False,False,False,1,0.246637,0.361045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,9995,False,False,jComfqYTXYSeH3GvqcOhwPmldb+BCdVJDKKDNkdtw2w=,Kr2CC15nSwDjdpyAeVh4vOuIaHuC/Q7cL9BAK28JoG8=,1.207136,0.082855,0.918960,0.313880,0.495189,...,0.0,0.810,4677,3307,True,False,True,1,0.502268,0.486637
9995,9996,False,False,Pr5enXjzVzVjZziZxrDcgnyu6CLUftmEbnp6TctyJbU=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,1.414798,0.000000,1.000000,0.000000,0.357369,...,0.0,1.000,1262,892,True,False,True,15,0.890135,0.346276
9996,9997,True,True,9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,1.413677,0.000000,1.000000,0.000000,0.668517,...,0.0,1.000,1261,892,True,False,True,5,0.726457,0.659001
9997,9998,False,False,9zkXU3f6YnRPjsWi3lKSCLseIGrleg00tpRI4OplABw=,gOZBAoajyr6i7GgON0N7q5+KE4JTwH3OUM0lZOWMuG8=,1.294118,0.000000,1.000000,0.000000,0.570707,...,1.0,1.000,1188,918,False,False,False,3,0.450980,0.561448


In [ ]:
df.dtypes

id        int64
x1         bool
x2         bool
x3       object
x4       object
         ...   
x141       bool
x142       bool
x143      int64
x144    float64
x145    float64
Length: 146, dtype: object

In [ ]:
df12.x3.nunique()

66

In [ ]:
df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [ ]:
df.drop(columns=['id'], inplace=True)

In [ ]:
 # Check for missing values in object columns, considering only object columns
object_cols = df.select_dtypes(include=['O']).columns
object_cols_with_missing = [col for col in object_cols if df[col].isnull().any()]

# Forward fill missing values in object columns
df[object_cols_with_missing] = df[object_cols_with_missing].fillna(method='ffill')

df

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1,False,False,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.576561,0.073139,0.481394,0.115697,0.472474,...,0.0,0.810,3306,4676,True,False,True,2,0.375535,0.464610
1,2,True,True,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.510,4678,3306,True,False,True,4,0.741682,0.593630
2,3,False,False,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.341803,0.051422,0.935572,0.041440,0.501710,...,0.0,0.850,4678,3306,False,False,False,1,0.776467,0.493159
3,4,True,False,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,0.653912,0.041471,0.940787,0.090851,0.556564,...,0.0,0.945,3306,4678,False,False,True,3,0.168234,0.546582
4,5,False,False,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,1.415919,0.000000,1.000000,0.000000,0.375297,...,0.0,1.000,1263,892,False,False,False,1,0.246637,0.361045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,9995,False,False,jComfqYTXYSeH3GvqcOhwPmldb+BCdVJDKKDNkdtw2w=,Kr2CC15nSwDjdpyAeVh4vOuIaHuC/Q7cL9BAK28JoG8=,1.207136,0.082855,0.918960,0.313880,0.495189,...,0.0,0.810,4677,3307,True,False,True,1,0.502268,0.486637
9995,9996,False,False,Pr5enXjzVzVjZziZxrDcgnyu6CLUftmEbnp6TctyJbU=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,1.414798,0.000000,1.000000,0.000000,0.357369,...,0.0,1.000,1262,892,True,False,True,15,0.890135,0.346276
9996,9997,True,True,9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,1.413677,0.000000,1.000000,0.000000,0.668517,...,0.0,1.000,1261,892,True,False,True,5,0.726457,0.659001
9997,9998,False,False,9zkXU3f6YnRPjsWi3lKSCLseIGrleg00tpRI4OplABw=,gOZBAoajyr6i7GgON0N7q5+KE4JTwH3OUM0lZOWMuG8=,1.294118,0.000000,1.000000,0.000000,0.570707,...,1.0,1.000,1188,918,False,False,False,3,0.450980,0.561448


In [ ]:
# seperating the object column in new df named object_df
object_df = df.select_dtypes(include=['object'])
object_df


,x3,x4,x34,x35,x61,x64,x65,x91,x94,x95
0,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,mimucPmJSF6NI6KM6cPIaaVxWaQyIQzSgtwTTb9bKlc=,s7mTY62CCkWUFc36AW2TlYAy5CIcniD2Vz+lHzyYCLg=,+2TNtXRI6r9owdGCS80Ia9VVv8ZpuOpVaHEvxRGGu78=,Op+X3asn5H7EQJErI7PR0NkUs3YB+Ld/8OfWuiOC8tU=,GeerC2BbPUcQfQO86NmvOsKrfTvmW7HF+Iru9y+7DPA=,bxU52teuxC05EZyzFihSiKHczE2ZAIVCXekVLG7j3C0=,+dia7tCOijlRGbABX0YKG5L85x/hXLyJwwplN5Qab04=,f4Uu1R9nnf/h03aqiRQT0Fw3WItzNToLCyRlW1Pn8Z8=
1,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,l0G2rvmLGE6mpPtAibFsoW/0SiNnAuyAc4k35TrHvoQ=,lblNNeOLanWhqgISofUngPYP0Ne1yQv3QeNHqCAoh48=,MZZbXga8gvaCBqWpzrh2iKdOkcsz/bG/z4BVjUnqWT0=,TqL9cs8ZFzALzVpZv6wYBDi+6zwhrdarQE/3FH+XAlA=,aZTF/lredyP4cukeN8bh6kpBjYmS1QFNpPOg2LVm3Lg=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,dsyhxXKNNJy4WVGD/v4+UGyW3jHWkx2xTdg3STsf34A=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=
2,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,9TRXThP/ifDpJRGFX1LQseibUA1NJ3XM53gy+1eZ46k=,XSJ6E8aAoZC7/KAu3eETpfMg3mCq7HVBFIVIsoMKh9E=,cr+kkNnNFV9YL0vz029hk3ohIDmGuABRVNhFe0ePZyo=,oFsUwSLCWcj8UA1cqILh5afKVcvwlFA+ohJ147Wkz5I=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,mRPnGiKVOWTk/vzZaqlLXZRtdrkcQ/sX0hqBCqOuKq0=,oo9tGpHvTredpg9JkHgYbZAuxcwtSpQxU5mA/zUbxY8=
3,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,XQG0f+jmjLI0UHAXXH2RYL4MEHa+yd9okO+730PCZuc=,/1yAAEg6Qib4GMD+wvGOlGmpCIPIAzioWtcCwbns9/I=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,Vl+TDNSupucNoI+Fqeo7bMCkxg1hRjgTSS6NYb9BW00=,/1yAAEg6Qib4GMD+wvGOlGmpCIPIAzioWtcCwbns9/I=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=
4,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,XEDyQD4da6aJkZiBf+r7LD2VdhLGnCMsSpuRFUyCZgg=,Co/nVSLofrWsM5qpcKLXfekegArokgN29XjEXttuXK4=,wIHg6aGH2GMPX6l1pCTzeS1bXE4jxRqmd9ubES4HgW8=,ST8+q2Jgb91pWEwLwmSoJzXEGsQKeQGbzlLbgHPtj4w=,rB07AAHPffU4zFFF8IrqfKSltyWcPyy4+q+IM5SLZiQ=,WYQEP5EEzM+P+nfkHKLkGko/S3RdBgfEQ3IcyYwrChE=,fylJzYvYlM0+kRBeLB3eFKKgCibqxFvBa8hL+WStwCE=,IoM2E9pNxABFR+H3yfapUL+ThKm7GtTzY7js9H/H99o=
...,...,...,...,...,...,...,...,...,...,...
9994,jComfqYTXYSeH3GvqcOhwPmldb+BCdVJDKKDNkdtw2w=,Kr2CC15nSwDjdpyAeVh4vOuIaHuC/Q7cL9BAK28JoG8=,SRFkJlXGZgnI2svGQLoCAcrghqMRr+u5s36xzSMAOqg=,ubHy/++3FVAS97znZOt7L+cjkZFJREIiJPRZEfRIztc=,7uf0BQkpKFCgsoTY6hGENDudghJBAtKvDQ3VTc1nO7E=,lPUeS6siL3Hb9UUwnRC9piF2fYeBf+u85lUSgk4qgg4=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,NMZ5RsVp7WQ9rJP2seQnLYgSaJ7ga0FV3Ieg0DW59C0=,2QaQ5ANfKi6SrWtkIv5y7DUAmEz3dJXLn5fhhjBV8N8=,uO9xKdGEeEsG0BVm6VI/0XYd8E0DRuXXEa2gwcoKHcg=
9995,Pr5enXjzVzVjZziZxrDcgnyu6CLUftmEbnp6TctyJbU=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,v4GptRPKxsWXk7yKmmdhHBTqN7QoYJTK1GPjpjppMa4=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,gtvCdEuc1Tnjv6MSRDG8mAMO+KHeyqX/rg0IQwpdbi8=,v4GptRPKxsWXk7yKmmdhHBTqN7QoYJTK1GPjpjppMa4=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,zX9+hre+RQeHdvHyFAguXw2WNsshYzygopGqPn/BDLc=,v4GptRPKxsWXk7yKmmdhHBTqN7QoYJTK1GPjpjppMa4=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=
9996,9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,9cgVS5E58bStBXAoRa9+MN4C7HpJh+UfM6/QcCwH0k4=,FExKgjj6CsbToTubdZ+kGsOmUx3gCvZVJCdZPcdPNF4=,9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=,c5oB4c9pSRTzkd4PmhzY4BazFbmVHbhy0lbyPp7aRbA=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,zOxT5daF0yUlUsnKTZeXgFCmbLqtE7oBITbptqroU/Q=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=
9997,9zkXU3f6YnRPjsWi3lKSCLseIGrleg00tpRI4OplABw=,gOZBAoajyr6i7GgON0N7q5+KE4JTwH3OUM0lZOWMuG8=,lnHueTbAHnZiCYqByDCzhzcB9kjjnk6GOmLFNK8gG00=,Kr2CC15nSwDjdpyAeVh4vOuIaHuC/Q7cL9BAK28JoG8=,zsF/C4x766PfoC59pZccSIWFOtQtiX/RPXB76PwIvIg=,lnHueTbAHnZiCYqByDCzhzcB9kjjnk6GOmLFNK8gG00=,Kr2CC15nSwDjdpyAe

In [ ]:
# Get the column names
column_names = object_df.columns

# Print or use the column names as needed
print("Column Names:", column_names)

Column Names: Index(['x3', 'x4', 'x34', 'x35', 'x61', 'x64', 'x65', 'x91', 'x94', 'x95'], dtype='object')


In [ ]:
# Concatenating object columns into a new single column
new_df = object_df.apply(lambda x: '_'.join(x), axis=1).to_frame(name='concat_col')

print(new_df)


                                             concat_col
0     dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=_G...
1     dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=_G...
2     ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=_X...
3     BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=_Y...
4     RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=_3...
...                                                 ...
9994  jComfqYTXYSeH3GvqcOhwPmldb+BCdVJDKKDNkdtw2w=_K...
9995  Pr5enXjzVzVjZziZxrDcgnyu6CLUftmEbnp6TctyJbU=_Y...
9996  9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=_W...
9997  9zkXU3f6YnRPjsWi3lKSCLseIGrleg00tpRI4OplABw=_g...
9998  MZZbXga8gvaCBqWpzrh2iKdOkcsz/bG/z4BVjUnqWT0=_Y...

[9999 rows x 1 columns]


### **use of cryptic hash**

In [ ]:

def generate_hash(text):

    # Encode the text to bytes
    text_bytes = text.encode('utf-8')

    # Create a SHA-256 hash object
    hash_object = hashlib.sha256()

    # Update the hash object with the text bytes
    hash_object.update(text_bytes)

    # Get the hexadecimal representation of the hash
    hash_hex = hash_object.hexdigest()

    return hash_hex

# Example usage

new_df['hashed_col8'] = new_df['concat_col'].apply(generate_hash)




In [ ]:
new_df

,concat_col,hashed_col8
0,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=_G...,c25de787fb13fbba0826ff7d35b97b694361108f63cd8f...
1,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=_G...,34e022778369cd9f1f53f946ae2d55b3b05494c6fb5a95...
2,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=_X...,974dd6e2fc1cd6de998113ecef9149c0bccd54d8f607f1...
3,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=_Y...,a69dd471b7422a1504e3aa5367f5f4d95c28a08d78deb8...
4,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=_3...,2c2ae2f6abc0227134afe4299fb707e308ac933b24b0ce...
...,...,...
9994,jComfqYTXYSeH3GvqcOhwPmldb+BCdVJDKKDNkdtw2w=_K...,e707c4126cb743ae7ea5b3a71c5369edadeef947d33257...
9995,Pr5enXjzVzVjZziZxrDcgnyu6CLUftmEbnp6TctyJbU=_Y...,605081ba4fc2c048f9918502aab5e80c43b4d48b79164b...
9996,9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=_W...,453e215acb858b76dc3138cb233e1352ca2c12f4859fc9...
9997,9zkXU3f6YnRPjsWi3lKSCLseIGrleg00tpRI4OplABw=_g...,a8292f0b637e198c97dfa2f90520b3186abbd19ce675fe...


In [ ]:
new_df.dtypes

concat_col    object
dtype: object

In [ ]:
object_df

,x3,x4,x34,x35,x61,x64,x65,x91,x94,x95
0,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,mimucPmJSF6NI6KM6cPIaaVxWaQyIQzSgtwTTb9bKlc=,s7mTY62CCkWUFc36AW2TlYAy5CIcniD2Vz+lHzyYCLg=,+2TNtXRI6r9owdGCS80Ia9VVv8ZpuOpVaHEvxRGGu78=,Op+X3asn5H7EQJErI7PR0NkUs3YB+Ld/8OfWuiOC8tU=,GeerC2BbPUcQfQO86NmvOsKrfTvmW7HF+Iru9y+7DPA=,bxU52teuxC05EZyzFihSiKHczE2ZAIVCXekVLG7j3C0=,+dia7tCOijlRGbABX0YKG5L85x/hXLyJwwplN5Qab04=,f4Uu1R9nnf/h03aqiRQT0Fw3WItzNToLCyRlW1Pn8Z8=
1,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,l0G2rvmLGE6mpPtAibFsoW/0SiNnAuyAc4k35TrHvoQ=,lblNNeOLanWhqgISofUngPYP0Ne1yQv3QeNHqCAoh48=,MZZbXga8gvaCBqWpzrh2iKdOkcsz/bG/z4BVjUnqWT0=,TqL9cs8ZFzALzVpZv6wYBDi+6zwhrdarQE/3FH+XAlA=,aZTF/lredyP4cukeN8bh6kpBjYmS1QFNpPOg2LVm3Lg=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,dsyhxXKNNJy4WVGD/v4+UGyW3jHWkx2xTdg3STsf34A=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=
2,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,9TRXThP/ifDpJRGFX1LQseibUA1NJ3XM53gy+1eZ46k=,XSJ6E8aAoZC7/KAu3eETpfMg3mCq7HVBFIVIsoMKh9E=,cr+kkNnNFV9YL0vz029hk3ohIDmGuABRVNhFe0ePZyo=,oFsUwSLCWcj8UA1cqILh5afKVcvwlFA+ohJ147Wkz5I=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,mRPnGiKVOWTk/vzZaqlLXZRtdrkcQ/sX0hqBCqOuKq0=,oo9tGpHvTredpg9JkHgYbZAuxcwtSpQxU5mA/zUbxY8=
3,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,XQG0f+jmjLI0UHAXXH2RYL4MEHa+yd9okO+730PCZuc=,/1yAAEg6Qib4GMD+wvGOlGmpCIPIAzioWtcCwbns9/I=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,Vl+TDNSupucNoI+Fqeo7bMCkxg1hRjgTSS6NYb9BW00=,/1yAAEg6Qib4GMD+wvGOlGmpCIPIAzioWtcCwbns9/I=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=
4,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,XEDyQD4da6aJkZiBf+r7LD2VdhLGnCMsSpuRFUyCZgg=,Co/nVSLofrWsM5qpcKLXfekegArokgN29XjEXttuXK4=,wIHg6aGH2GMPX6l1pCTzeS1bXE4jxRqmd9ubES4HgW8=,ST8+q2Jgb91pWEwLwmSoJzXEGsQKeQGbzlLbgHPtj4w=,rB07AAHPffU4zFFF8IrqfKSltyWcPyy4+q+IM5SLZiQ=,WYQEP5EEzM+P+nfkHKLkGko/S3RdBgfEQ3IcyYwrChE=,fylJzYvYlM0+kRBeLB3eFKKgCibqxFvBa8hL+WStwCE=,IoM2E9pNxABFR+H3yfapUL+ThKm7GtTzY7js9H/H99o=
...,...,...,...,...,...,...,...,...,...,...
9994,jComfqYTXYSeH3GvqcOhwPmldb+BCdVJDKKDNkdtw2w=,Kr2CC15nSwDjdpyAeVh4vOuIaHuC/Q7cL9BAK28JoG8=,SRFkJlXGZgnI2svGQLoCAcrghqMRr+u5s36xzSMAOqg=,ubHy/++3FVAS97znZOt7L+cjkZFJREIiJPRZEfRIztc=,7uf0BQkpKFCgsoTY6hGENDudghJBAtKvDQ3VTc1nO7E=,lPUeS6siL3Hb9UUwnRC9piF2fYeBf+u85lUSgk4qgg4=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,NMZ5RsVp7WQ9rJP2seQnLYgSaJ7ga0FV3Ieg0DW59C0=,2QaQ5ANfKi6SrWtkIv5y7DUAmEz3dJXLn5fhhjBV8N8=,uO9xKdGEeEsG0BVm6VI/0XYd8E0DRuXXEa2gwcoKHcg=
9995,Pr5enXjzVzVjZziZxrDcgnyu6CLUftmEbnp6TctyJbU=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,v4GptRPKxsWXk7yKmmdhHBTqN7QoYJTK1GPjpjppMa4=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,gtvCdEuc1Tnjv6MSRDG8mAMO+KHeyqX/rg0IQwpdbi8=,v4GptRPKxsWXk7yKmmdhHBTqN7QoYJTK1GPjpjppMa4=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,zX9+hre+RQeHdvHyFAguXw2WNsshYzygopGqPn/BDLc=,v4GptRPKxsWXk7yKmmdhHBTqN7QoYJTK1GPjpjppMa4=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=
9996,9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,9cgVS5E58bStBXAoRa9+MN4C7HpJh+UfM6/QcCwH0k4=,FExKgjj6CsbToTubdZ+kGsOmUx3gCvZVJCdZPcdPNF4=,9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=,c5oB4c9pSRTzkd4PmhzY4BazFbmVHbhy0lbyPp7aRbA=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,zOxT5daF0yUlUsnKTZeXgFCmbLqtE7oBITbptqroU/Q=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=
9997,9zkXU3f6YnRPjsWi3lKSCLseIGrleg00tpRI4OplABw=,gOZBAoajyr6i7GgON0N7q5+KE4JTwH3OUM0lZOWMuG8=,lnHueTbAHnZiCYqByDCzhzcB9kjjnk6GOmLFNK8gG00=,Kr2CC15nSwDjdpyAeVh4vOuIaHuC/Q7cL9BAK28JoG8=,zsF/C4x766PfoC59pZccSIWFOtQtiX/RPXB76PwIvIg=,lnHueTbAHnZiCYqByDCzhzcB9kjjnk6GOmLFNK8gG00=,Kr2CC15nSwDjdpyAe

### **USE of bag of words**

In [ ]:
data = new_df['concat_col']

# Create CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the data
X = vectorizer.fit_transform(data)

# Get feature names (words)
feature_names = vectorizer.get_feature_names_out()

# Convert to DataFrame (optional)
Cv_df = pd.DataFrame(X.toarray(), columns=feature_names)

Cv_df


,00b0w,00cweqs5zc22wnxklsmc6aqquav65,00lqqxw4ilqbljm,00oox76q12trsmrswd,00p4mhwehcb4ktx56mmfdyykiwaqneudcc7mhwqpkei,012jgeckdldu,01fua6sfh369yiv9fv,01ic858dwxr6ea3xzgboaxa54ct9i,01ipzreo6eyu,01isp1hk2dchkxxe8kjp,...,zztmh5tvf9pqeukjm6wviflaj0olcree,zzu2hq6npznpd0e4,zzuffgh9y3ofgvrfdlirfsvnti1jmeux7zv3imp8rjc,zzun0,zzvc2ccbuh,zzvvzf1kjg42qllqxp5fhnd0,zzwbgwrqdlys1hm,zzweas6se23crgbsguzmx3ndgd2wo8nwre,zzz6jtw3vug,zzzjmqhhm
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### **Using Tf IDF vectoriser**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Create TF-IDF vectorizer instance
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the corpus and transform the corpus
tfidf_matrix = vectorizer.fit_transform(data)

# Get feature names (terms)
feature_names = vectorizer.get_feature_names_out()

# Convert sparse matrix to dense matrix for representation (not typically done for large datasets)
dense_tfidf_matrix = tfidf_matrix.toarray()

# Print TF-IDF matrix
print("TF-IDF Matrix:")
print(dense_tfidf_matrix)

# Print feature names
print("\nFeature Names (terms):")
print(feature_names)


TF-IDF Matrix:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Feature Names (terms):
['00b0w' '00bam' '00co4tz47lw8x' ...
 'zzzdxdo40ouovbagyd4or6wfmjvmg0wsbgumqaatqms' 'zzzjmqhhm' 'zzzucsq8b']


### **Word Embedding**

In [ ]:
from gensim.models import KeyedVectors
import numpy as np

# Load pre-trained Word2Vec embeddings
# Replace 'actual/path/to/word2vec.bin' with the correct file path
path_to_word2vec_file = 'actual/path/to/word2vec.bin'
word_vectors = KeyedVectors.load_word2vec_format(path_to_word2vec_file, binary=True)

# Example documents
documents = data

# Function to preprocess text (tokenization, lowercasing, etc.)
def preprocess(text):
    return text.lower().split()

# Function to generate document embeddings
def document_embedding(doc_tokens, word_vectors):
    # Initialize an empty list to store embeddings
    embeddings = []
    for token in doc_tokens:
        if token in word_vectors:
            embeddings.append(word_vectors[token])
    # Return mean of embeddings as document embedding
    return np.mean(embeddings, axis=0)

# Calculate document embeddings
document_embeddings = []
for doc in documents:
    tokens = preprocess(doc)
    doc_embedding = document_embedding(tokens, word_vectors)
    document_embeddings.append(doc_embedding)

# Print document embeddings
for i, embedding in enumerate(document_embeddings):
    print(f"Document {i+1} embedding:")
    print(embedding)
    print()

FileNotFoundError: [Errno 2] No such file or directory: 'actual/path/to/word2vec.bin'